In [3]:
import pandas as pd 
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
import re
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.utils import to_categorical 
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential 
from keras.layers import LSTM, Dense, GRU, Embedding 
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [4]:
import numpy as np

data = pd.read_csv('./text_emotion.csv')

# Let's drop unnecessary columns from our dataset
data = data.drop('tweet_id', axis=1)
data = data.drop('author', axis=1)
data.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [5]:
data['sentiment'].unique()


array(['empty', 'sadness', 'enthusiasm', 'neutral', 'worry', 'surprise',
       'love', 'fun', 'hate', 'happiness', 'boredom', 'relief', 'anger'],
      dtype=object)

In [6]:
data = data.drop(data[data.sentiment == 'boredom'].index)
data = data.drop(data[data.sentiment == 'surprise'].index)
data = data.drop(data[data.sentiment == 'enthusiasm'].index)
data = data.drop(data[data.sentiment == 'empty'].index)
data = data.drop(data[data.sentiment == 'fun'].index)
data = data.drop(data[data.sentiment == 'relief'].index)
data = data.drop(data[data.sentiment == 'love'].index)
data = data.drop(data[data.sentiment == 'neutral'].index)

In [7]:
data['sentiment'].replace(to_replace='hate', value='anger', inplace=True)
# Let's replace 'worry' for 'fear'
data['sentiment'].replace(to_replace='worry', value='fear', inplace=True)
# Let's see how many tweets we have for each sentiment
data.groupby('sentiment')['sentiment'].count().sort_values(ascending=False)

sentiment
fear         8459
happiness    5209
sadness      5165
anger        1433
Name: sentiment, dtype: int64

In [8]:
from nltk.corpus import wordnet
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')
import random

def get_synonyms(word):
    """
    Get synonyms of a word
    """
    synonyms = set()
    
    for syn in wordnet.synsets(word): 
        for l in syn.lemmas(): 
            synonym = l.name().replace("_", " ").replace("-", " ").lower()
            synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
            synonyms.add(synonym) 
    
    if word in synonyms:
        synonyms.remove(word)
    
    return list(synonyms)
def synonym_replacement(words, n):
    
    words = words.split()
    
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in stop]))
    random.shuffle(random_word_list)
    num_replaced = 0
    
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        
        if num_replaced >= n: #only replace up to n words
            break
        return sentence

# Random Insertion

def random_insertion(words, n):
    
    words = words.split()
    new_words = words.copy()
    
    for _ in range(n):
        add_word(new_words)
        
    sentence = ' '.join(new_words)
    return sentence

def add_word(new_words):
    
    synonyms = []
    counter = 0
    
    while len(synonyms) < 1:
        random_word = new_words[random.randint(0, len(new_words)-1)]
        synonyms = get_synonyms(random_word)
        counter += 1
        if counter >= 10:
            return
        
    random_synonym = synonyms[0]
    random_idx = random.randint(0, len(new_words)-1)
    new_words.insert(random_idx, random_synonym)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
#Making all letters lowercase
data['content'] = data['content'].apply(lambda x: " ".join(x.lower() for x in x.split()))

#Removing Punctuation, Symbols
data['content'] = data['content'].str.replace('[^\w\s]',' ')

#Removing urls, links
data['content'] = data['content'].str.replace('(www|http)\S+', ' ')

#Removing Stop Words using NLTK
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['content'] = data['content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))



In [11]:
nltk.download('wordnet')
p = stopwords.words('english')
data['content'] = data['content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#Lemmatisation
from textblob import Word
data['content'] = data['content'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

#Correcting Letter Repetitions
import re

def de_repeat(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

data['content'] = data['content'].apply(lambda x: " ".join(de_repeat(x) for x in x.split()))




[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [12]:
# Code to find the top 1000 rarest words appearing in the data
freq = pd.Series(' '.join(data['content']).split()).value_counts()[-1000:]

# Removing all those rarely appearing words from the data
freq = list(freq.index)
data['content'] = data['content'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
print(freq)
print(data['content'])


['ohbabayy33', 'bevjack', 'have2live', 'ruthiexo09', 'vacuumed', 'vado', 'aerinea', 'eb', 'afaik', 'cutebug', 'babblingbrookie', 'mymea', 'fcb', 'phï', 'carlocruuz', 'gh_confidential', 'fence', 'rosa', 'flightwas', 'debindenver', 'dealer', 'rgoodchild', 'punjab', 'celebritytweet', 'viveknshah', 'lovelauriexox', 'areo', 'me215', 'andreshenriquez', 'n00rtje', 'jennycliu', '4wh0o', 'helsinki', '679tn', 'retreat', 'arik', 'headlight', 'scrapshoppeetsy', 'woowoo', 'piglet', 'afceahelen', 'baka', 'confuzzledd', 'mpg', 'vincew', 'scheuring', '20am', 'cooperated', 'tos', 'tom_conrad', 'wormie', '821', 'raha', 'aamazing', 'invincible', 'meself', 'karenbrady', 'dion', 'lightsresolve', 'bcoz', 'ttytomorrow', 'beshirthappy', 'gronumbulator', 'karlixpaz', 'dee_staack', 'datachick', 'ethic', 'laurenkel', 'cassone', 'qiuqiu', 'cj', 'samuri', 'stylenista', 'jerseyzmorenita', 'eddieishere', 'bournville', 'ihuntmidgets', 'gillian9', 'triplet', 'peac', 'terence', 'sprinkle', 'kenduri', 'stayingyoung', 's

In [13]:
data.head()

,sentiment,content
1,sadness,layin n bed headache ughh waitin call
2,sadness,funeral ceremony gloomy friday
5,fear,pinging ghostridah14 go prom bc bf like friend
6,sadness,sleep im thinking old friend want married damn...
7,fear,hmm http ww djhero com


In [14]:

import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In [17]:
#Encoding output labels
from sklearn import preprocessing
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(data.sentiment.values)

# Splitting into training and testing data in 90:10 ratio
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(data.content.values, y, stratify=y, random_state=42, test_size=0.1, shuffle=True)


In [19]:

y_train

array([3, 3, 1, ..., 2, 2, 1])

In [20]:
X_train

array(['headache want go think worth maybe know',
       'favorite spotter last tweet go phone',
       'passed 700 00 view youtube video', ...,
       'okayy mail thenn lol im much candy amp amp jello x',
       'loebette hooray avatar back wil go put zone',
       'twilightgossip hope feel better soon'], dtype=object)

In [27]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)


sequencext = tokenizer.texts_to_sequences(X_train)
paddedxt = pad_sequences(sequencext, padding='post')
print(paddedxt[0])
print(paddedxt.shape) 
print(y_train.shape)

sequencexv = tokenizer.texts_to_sequences(X_val)
paddedxv = pad_sequences(sequencexv, padding='post')
print(paddedxv[0])
print(paddedxv.shape) 




[221  23   6  26 610 174  18   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0]
(18239, 26)
(18239,)
[ 233  257   85   33 5101  228    6 3033  182    1    1    0    0    0
    0    0    0    0    0    0    0    0    0]
(2027, 23)


In [ ]:
import numpy as np
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(25620, 16, input_length=120),

    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

num_epochs = 50
training_padded = paddedxt
training_labels = y_train
testing_padded = paddedxv
testing_labels =y_val
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=1)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 16)           409920    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 120, 128)          41472     
_________________________________________________________________
bidirectional_4 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense_4 (Dense)              (None, 24)                1560      
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 100       
Total params: 494,268
Trainable params: 494,268
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
570/570 [==============================] - 29s 52ms/step - loss: 2.1509 - accuracy: 0.3100 - val_loss: 